# ADMN5006 - Analytics for Financial Markets

## Assignment #1

<p>Your assignment is to design and develop machine learning models for corporate bankruptcy prediction 2 years into the future using the data provided. </p>
   - The column “BK” in the data provided denotes whether the company goes bankrupt (indicated by 1) 2 years in the future, or not (indicated by 0). 
   - Some of the input variables included are “Assets Growth”, “Sales Growth”, “Earnings-per-share (EPS)”, “Return-on-equity (ROE)”, etc. 
   - In addition, the paper entitled: “Machine learning models and bankruptcy prediction” by Barboza et. al. is posted to the portal and is an excellent reference for you to use.  
    
<p>Note however that the dataset you are given is not the same as the one used in the aforementioned paper.  Therefore, don’t be alarmed if your results don’t agree with the Barboza et. al. paper.  You will need to partition your data into testing and training sets and explain your experimental set-up.</p>

<p>The input and output variables are described below:</p>

- EPS – Earnings Per Share
- Liquidity – Working Capital/Total Assets
- Profitability – Retained Earnings/Total Assets
- Productivity – EBIT/Total Assets
- Leverage Ratio – (Total Long-term debt + Debt in Current liabilities)/Stockholders Equity
- Asset Turnover – Sales/ Total Assets
- Operational Margin – EBIT/Sales
- Market Book Ratio – (Price Close Annual Fiscal * Common Shares Outstanding)/Book Value Per Share
- Asset Growth – Change in assets from previous year
- Sales Growth – Change in sales from previous year
- Employee Growth – Change in employees from previous year
- Tobin’s Q – (Total market value of company + liabilities)/ (Total asset or book value + liabilities)
- BK – Company bankrupt or not


    

## 1. Stage One

### 1.1 Inventory resources
    - Personnel: Luke, Jad, Rita, Sreehari, Arnab, Varun
    - Data: Banruptcy_data_Final
    - Computing resources:  Machine Learning (Random Forest, SVC, Naive Bayes)
    - Software: Jupyter Notebook

## 2. Stage Two

### 2.1 Initial Data Report

In [66]:
# Import Libraries Required
#working with the data
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns

#for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#computing resources
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [5]:
#Data source: 
#Source Query location: 
path =  'Bankruptcy_data_Final.xlsx'
# reads the data from the file - denotes as CSV, it has no header, sets column headers
#df =  pd.read_csv(path, sep=',')
df = pd.read_excel(path)

### 2.2 Describe Data

In [7]:
df.columns

Index(['EPS', 'Liquidity', 'Profitability', 'Productivity', 'Leverage Ratio',
       'Asset Turnover', 'Operational Margin', 'Return on Equity',
       'Market Book Ratio', 'Assets Growth', 'Sales Growth', 'Employee Growth',
       'BK'],
      dtype='object')

In [36]:
df.shape

(92872, 13)

In [37]:
df.dtypes

EPS                   float64
Liquidity             float64
Profitability         float64
Productivity          float64
Leverage Ratio        float64
Asset Turnover        float64
Operational Margin    float64
Return on Equity      float64
Market Book Ratio     float64
Assets Growth         float64
Sales Growth          float64
Employee Growth       float64
BK                      int64
dtype: object

In [8]:
df.describe()

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
count,92867.000000,92625.000000,92625.000000,92625.000000,92846.000000,92625.000000,87315.000000,92864.000000,9.281500e+04,86171.000000,86171.000000,85862.000000,92872.000000
mean,-14.461355,-2.631237,-29.537274,-1.222870,1.345316,1.053613,-7.915485,-2.112382,3.585063e+02,1.294075,1.900108,0.343322,0.006008
std,2195.467288,121.610921,677.230667,35.885556,253.038093,2.115945,214.460079,352.596902,2.606364e+04,73.769522,177.632638,14.074156,0.077280
min,-384000.000000,-25968.520000,-79682.000000,-5093.000000,-7811.000000,-31.590000,-30175.700000,-88875.140000,-3.151500e+06,-1.000000,-27.431034,-1.000000,0.000000
25%,-0.140000,0.020000,-0.640000,-0.060000,0.000000,0.390000,-0.030000,-0.080000,1.120000e+01,-0.052596,-0.033522,-0.048107,0.000000
50%,0.330000,0.190000,0.070000,0.060000,0.280000,0.830000,0.060000,0.030000,5.828000e+01,0.051844,0.059733,0.017241,0.000000
75%,1.530000,0.400000,0.310000,0.110000,0.820000,1.390000,0.140000,0.070000,2.401400e+02,0.192062,0.204231,0.130944,0.000000
max,55339.000000,1.000000,140.580000,1102.000000,75970.380000,276.380000,394.470000,39500.000000,3.455419e+06,14231.000000,39850.000000,2699.000000,1.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92872 entries, 0 to 92871
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EPS                 92867 non-null  float64
 1   Liquidity           92625 non-null  float64
 2   Profitability       92625 non-null  float64
 3   Productivity        92625 non-null  float64
 4   Leverage Ratio      92846 non-null  float64
 5   Asset Turnover      92625 non-null  float64
 6   Operational Margin  87315 non-null  float64
 7   Return on Equity    92864 non-null  float64
 8   Market Book Ratio   92815 non-null  float64
 9   Assets Growth       86171 non-null  float64
 10  Sales Growth        86171 non-null  float64
 11  Employee Growth     85862 non-null  float64
 12  BK                  92872 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 9.2 MB


In [40]:
df.head(5)

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
0,1.58,0.36,0.18,0.13,1.33,1.77,0.07,0.15,2.22,NaN,NaN,NaN,0
1,1.41,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126319,0.014278,0.040179,0
2,0.31,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368077,0.327909,0.566524,0
3,0.71,0.28,0.14,0.08,0.80,1.39,0.06,0.05,5.28,-0.020809,-0.118904,-0.095890,0
4,0.75,0.41,0.13,0.08,0.20,1.30,0.06,0.04,8.68,0.233089,0.146807,0.053030,0


### 2.3 Check Data Validity

#### 2.3.1 Check Missing Data

In [41]:
df.isnull().sum()

EPS                      5
Liquidity              247
Profitability          247
Productivity           247
Leverage Ratio          26
Asset Turnover         247
Operational Margin    5557
Return on Equity         8
Market Book Ratio       57
Assets Growth         6701
Sales Growth          6701
Employee Growth       7010
BK                       0
dtype: int64

In [42]:
def missing_values_table(df):
    #Code Description: Attempt to show the missing values in the table
    #Input: Dataframe
    #Output: missing value table rendered columns
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [43]:
missing_values_table(df) #it really depends

Your selected dataframe has 13 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
Employee Growth,7010,7.5
Assets Growth,6701,7.2
Sales Growth,6701,7.2
Operational Margin,5557,6.0
Liquidity,247,0.3
Profitability,247,0.3
Productivity,247,0.3
Asset Turnover,247,0.3
Market Book Ratio,57,0.1
Leverage Ratio,26,0.0


In [44]:
# Get the columns with > 50% missing
missing_df = missing_values_table(df);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('We will remove %d columns.' % len(missing_columns))

Your selected dataframe has 13 columns.
There are 12 columns that have missing values.
We will remove 0 columns.


##### No need to drop columns in the Dataframe

#### 2.3.2 Check Outliers and remove them

In [45]:
# Task: Check for Outliers per Column and determine what to do with it.
# check: https://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm 
# for more information
#    Below the first quartile − 3 ∗ interquartile range
#    Above the third quartile + 3 ∗ interquartile range

In [11]:
def check_outliers(data):
    outliers = []
    threshold = 3
    mean_1 = np.mean(data)
    std_1 = np.std(data)
    for y in data:
        z_score = (y - mean_1)/std_1
        if np.abs(z_score) > threshold:
            outliers.append(y)
    #print(outliers)
    print(f"There are {len(outliers)} number of outliers in column {df.column()}.")

In [18]:
def checkoutliers(data):
   return data[(np.abs(stats.zscore(df))<3).all(axis=1)]

In [30]:
#example
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK


In [ ]:
# pip install plotly --quiet
# pip install cufflinks --quiet

# if you haven't downloaded these 

In [ ]:
import plotly.express as px
import cufflinks as cf

df['Employee Growth'].iplot()

In [10]:
{column: len(df[column].unique()) for column in df.columns} # check if all are unique

{'EPS': 6988,
 'Liquidity': 6109,
 'Profitability': 9364,
 'Productivity': 6076,
 'Leverage Ratio': 6937,
 'Asset Turnover': 5052,
 'Operational Margin': 7486,
 'Return on Equity': 5751,
 'Market Book Ratio': 50337,
 'Assets Growth': 85605,
 'Sales Growth': 80351,
 'Employee Growth': 49533,
 'BK': 2}

All data are unique

## 2.4 Initial Data Exploration

### 2.41 Distributions

In [49]:
def count_values_table(df):
        count_val = df.value_counts()
        count_val_percent = 100 * df.value_counts() / len(df)
        count_val_table = pd.concat([count_val, count_val_percent.round(1)], axis=1)
        count_val_table_ren_columns = count_val_table.rename(
        columns = {0 : 'Count Values', 1 : '% of Total Values'})
        return count_val_table_ren_columns

In [50]:
# Histogram
def hist_chart(df, col):
        plt.style.use('fivethirtyeight')
        plt.hist(df[col].dropna(), edgecolor = 'k');
        plt.xlabel(col); plt.ylabel('Number of Entries');
        plt.title('Distribution of '+col);

In [ ]:
# This is just an Example
col = 'Employee Growth'
# Histogram & Results
hist_chart(df, col)
count_values_table(df.Liquidity)

### 2.4.2 Correlations

#### Differencing

In [66]:
# use seaborn and heatmap to show correlation

## 2.5 Data Quality Report

    -List the results of the data quality verification. If quality problems exist, suggest possible solutions. Solutions to data quality problems generally depend heavily on both data and business knowledge.

# Stage 3 - Data Preperation

In [52]:
# Initialize variables for Data inclusion/exclusion

## 3.2 Clean The Data

### 3.2.1 Label Encoding

In [53]:
# help? : http://pbpython.com/categorical-encoding.html

### 3.2.2 Drop Unnecessary Columns

In [54]:
# This is the code for deleting columns
# del_col_list = ['col1', 'col2']

# df = df.drop(del_col_list, axis=1)
# df.head()

### 3.2.3 Altering Data Types

In [55]:
#df['date'] = pd.to_datetime(df['date'])

###  3.2.4 Dealing With Zeros

In [56]:
#cols = ['col1', 'col2']
#df[cols] = df[cols].replace(0, np.nan)

In [57]:
# dropping all the rows with na in the columns mentioned above in the list.

# df.dropna(subset=cols, inplace=True)

### 3.2.5 Dealing With Duplications

In [58]:
#df = df.drop_duplicates(keep='first')

## 3.3 Construct Required Data

This task includes constructive data preparation operations such as the production of derived attributes or entire new records, or transformed values for existing attributes.

- Derived attributes - These are new attributes that are constructed from one or more existing attributes in the same record, for example you might use the variables of length and width to calculate a new variable of area.

- Generated records - Here you describe the creation of any completely new records. For example you might need to create records for customers who made no purchase during the past year. There was no reason to have such records in the raw data, but for modelling purposes it might make sense to explicitly represent the fact that particular customers made zero purchases.

In [56]:
def preprocess_inputs(df):
    df = df.copy()
    
    #split df into X and Y
    y = df['BK']
    X = df.drop('BK',axis =1)
    
    #train test split
    X_train,X_test,y_train,y_test= train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train),index=X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns = X_test.columns)
    return X_train, X_test, y_train, y_test

In [57]:
X_train, X_test, y_train, y_test = preprocess_inputs(df)

In [75]:
X_train

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth
6187,0.007440,0.027346,0.044540,0.037500,-0.006082,0.085240,0.036577,0.006600,-0.013806,-0.016418,-0.010600,-0.022845
92428,0.007647,0.022488,0.043908,0.038125,0.002307,-0.149966,0.036823,0.006694,-0.013566,-0.014325,-0.002002,-0.035093
80813,0.007212,-0.005964,-0.008864,0.004730,-0.006847,1.159386,0.035138,0.005578,-0.019546,-0.007940,0.003381,-0.028497
29167,0.006919,0.021608,0.043768,0.036050,0.002101,0.236722,0.036407,0.005721,-0.014060,-0.017105,-0.011097,-0.034193
17680,0.013464,0.023218,0.043933,0.036920,0.005228,-0.272809,0.036661,0.006671,-0.010291,-0.015226,-0.010138,-0.031703
...,...,...,...,...,...,...,...,...,...,...,...,...
21440,0.007549,0.024407,0.044250,0.037500,-0.005417,0.020975,0.036577,0.006576,-0.005735,-0.015071,-0.009992,-0.019302
73349,0.002588,0.022868,0.043699,0.038080,0.012546,0.796747,0.036534,0.000990,-0.013402,-0.018533,-0.011761,-0.038563
50057,0.007742,0.021188,0.044858,0.039530,-0.005417,-0.144279,0.037084,0.006600,-0.012982,-0.015335,-0.010481,-0.028497
5192,0.006105,0.025597,0.044002,0.035470,0.002433,0.167866,0.036365,0.005483,-0.014082,-0.014853,-0.010043,-0.034522


In [78]:
y_train

6187     0
92428    0
80813    0
29167    0
17680    0
        ..
21440    0
73349    0
50057    0
5192     0
77708    0
Name: BK, Length: 65010, dtype: int64

In [81]:
y_train

6187     0
92428    0
80813    0
29167    0
17680    0
        ..
21440    0
73349    0
50057    0
5192     0
77708    0
Name: BK, Length: 65010, dtype: int64

# Stage 4 - Exploratory Data Analysis
## Modelling

As the first step in modelling, you'll select the actual modelling technique that you'll be using. Although you may have already selected a tool during the business understanding phase, at this stage you'll be selecting the specific modelling technique e.g. decision-tree building with C5.0, or neural network generation with back propagation. If multiple techniques are applied, perform this task separately for each technique.

## 4.1 Modelling technique
Document the actual modelling technique that is to be used.
    
    Import Models Below:

In [82]:
models = {
    "Random Forest": RandomForestClassifier(),
    "SVC (RBF Kernel)": SVC(),
    "Naive Bayes" : GaussianNB()
}

## 4.2 Modelling Assumption
Many modelling techniques make specific assumptions about the data, for example that all attributes have uniform distributions, no missing values allowed, class attribute must be symbolic etc. Record any assumptions made.

## 4.3 Build Model
Run the modelling tool on the prepared dataset to create one or more models.

- Parameter settings - With any modelling tool there are often a large number of parameters that can be adjusted. List the parameters and their chosen values, along with the rationale for the choice of parameter settings.

- Models - These are the actual models produced by the modelling tool, not a report on the models.

- Model descriptions - Describe the resulting models, report on the interpretation of the models and document any difficulties encountered with their meanings.

In [ ]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name+ " trained.")

In [76]:
results = []
for name, model in models.items():
    result = model.score(X_test,y_test)
    results.append(result)
    print(name + ": {.2f}%".format(result * 100))
    

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 5.4 Assess Model 
<p>Interpret the models according to your domain knowledge, your data mining success criteria and your desired test design. Judge the success of the application of modelling and discovery techniques technically, then contact business analysts and domain experts later in order to discuss the data mining results in the business context. This task only considers models, whereas the evaluation phase also takes into account all other results that were produced in the course of the project.</p>

At this stage you should rank the models and assess them according to the evaluation criteria. You should take the business objectives and business success criteria into account as far as you can here. In most data mining projects a single technique is applied more than once and data mining results are generated with several different techniques.

- **Model assessment** - Summarise the results of this task, list the qualities of your generated models (e.g.in terms of accuracy) and rank their quality in relation to each other.

- **Revised parameter settings** - According to the model assessment, revise parameter settings and tune them for the next modelling run. Iterate model building and assessment until you strongly believe that you have found the best model(s). Document all such revisions and assessments.